In [5]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

In [6]:
presence_matrix = pd.read_csv(Path("../../data/crsp_raw") / "presence_matrix.csv")

In [7]:
jkp_data = pd.read_csv(Path("../../data/crsp_raw") / "crsp_all_data.csv")
jkp_data = jkp_data.rename(columns={c: c.lower() for c in jkp_data.columns})
jkp_data = jkp_data.dropna(subset=["permno"])
jkp_data["permno"] = jkp_data["permno"].astype(int)
jkp_data["date"] = pd.to_datetime(jkp_data["date"])
jkp_data = jkp_data.sort_values(["date", "permno"])
jkp_data = jkp_data.drop_duplicates(subset=["date", "permno"])
jkp_data = jkp_data.set_index(["date", "permno"])
jkp_data.head()

/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_74761/1091605028.py:1: DtypeWarning: Columns (5,6,9,18,19,20,22,24,38,40,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  jkp_data = pd.read_csv(Path("../../data/crsp_raw") / "crsp_all_data.csv")


nameendt  shrcd  exchcd siccd    ncusip ticker  \
date       permno                                                  
1980-01-02 10006       NaN     10       1  3743     80010    ACF   
           10057       NaN     11       1  3541  00462610    AMT   
           10058       NaN     10       3  5099  00077410   ABKC   
           10103       NaN     10       3  3496  00087410   ACSC   
           10137       NaN     11       1  4911  01741110    AYP   

                                        comnam shrcls tsymbol  naics  ...  \
date       permno                                                     ...   
1980-01-02 10006          A C F INDUSTRIES INC    NaN     NaN    NaN  ...   
           10057           ACME CLEVELAND CORP    NaN     NaN    NaN  ...   
           10058      A B K C O INDUSTRIES INC    NaN     NaN    NaN  ...   
           10103          A C S INDUSTRIES INC    NaN     NaN    NaN  ...   
           10137   ALLEGHENY POWER SYSTEMS INC    NaN     NaN    NaN  ...   

                  cfacpr cfacshr openprc  numtrd       retx    vwretd  \
date       permno                                                       
1980-01-02 10006     1.0     1.0     NaN     NaN   0.007299 -0.020089   
           10057     1.0     1.0     NaN     NaN  -0.004785 -0.020089   
           10058     1.0     1.0     NaN     NaN   0.000000 -0.020089   
           10103     1.0     1.0     NaN     NaN   0.166667 -0.020089   
           10137     2.0     2.0     NaN     NaN  -0.024194 -0.020089   

                     vwretx    ewretd    ewretx    sprtrn  
date       permno                                          
1980-01-02 10006  -0.020121 -0.011686 -0.011707 -0.020196  
           10057  -0.020121 -0.011686 -0.011707 -0.020196  
           10058  -0.020121 -0.011686 -0.011707 -0.020196  
           10103  -0.020121 -0.011686 -0.011707 -0.020196  
           10137  -0.020121 -0.011686 -0.011707 -0.020196  

[5 rows x 61 columns]

In [8]:
CRSP_IGNORED = [-66, -77, -88, -99]

jkp_data = jkp_data[
    (jkp_data["ret"] != CRSP_IGNORED[0])
    & (jkp_data["ret"] != CRSP_IGNORED[1])
    & (jkp_data["ret"] != CRSP_IGNORED[2])
    & (jkp_data["ret"] != CRSP_IGNORED[3])
]

In [9]:
jkp_data["ret"] = jkp_data["ret"].replace("C", np.nan).astype(float)

In [10]:
jkp_data["dolvol"] = jkp_data["vol"] * jkp_data["prc"]

In [11]:
MIN_OBS = 5

n_obs = jkp_data.groupby("permno").apply(lambda x: (x["prc"] >= 0).sum())
valid_ids = n_obs[n_obs > MIN_OBS].index

In [12]:
pivoted_returns = (
    jkp_data.loc[jkp_data.index.get_level_values("permno").isin(valid_ids)]
    .reset_index()
    .pivot_table(index="date", columns="permno", values="ret")
)

In [13]:
full_df = pivoted_returns

In [14]:
valid_cols = presence_matrix.columns.intersection(full_df.columns.astype(str))
len(valid_cols)

1457

In [15]:
presence_matrix = presence_matrix.rename(columns={"caldt": "date"})
presence_matrix["date"] = pd.to_datetime(presence_matrix["date"])
presence_matrix = presence_matrix.set_index("date")

In [16]:
merged_index = full_df.merge(
    presence_matrix, left_index=True, right_index=True, how="inner"
).index

In [17]:
full_df = full_df.loc[merged_index]
presence_matrix = presence_matrix.loc[merged_index]
full_df.shape, presence_matrix.shape

((11093, 20516), (11093, 1539))

In [18]:
full_df.columns = full_df.columns.astype(str)

In [19]:
full_df[valid_cols] = full_df[valid_cols] * presence_matrix[valid_cols]

In [20]:
full_df[valid_cols].to_csv(Path("../../data/output") / "spx_data.csv")

In [21]:
pd.DataFrame(valid_cols).to_csv(
    Path("../../data/output") / "spx_stocks_list.csv",
    index=False,
)